In [3]:
import os
import time
import shutil
import tarfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import seaborn as sns
import xgboost as xgb 
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import skew
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, r2_score
from sklearn.linear_model import Lasso, LassoCV, MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import optimizers
from keras.initializers import lecun_normal
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.neural_network import MLPRegressor

In [4]:
tf.__version__

'2.1.0'

In [6]:
# CSV TO DATA
def csvToData(csv_file):
    df = pd.read_csv(csv_file, header = 0)
    column = list(df.columns.values)
    data = df.values
    return column, data

sample_column, sample_csv_data = csvToData("data/sample_submission.csv")
test_column, test_csv_data = csvToData("data/test.csv")
train_column, train_csv_data = csvToData("data/train.csv")
print('sample_submission-----------')
print(sample_column)
print(sample_csv_data.shape)
print('test-----------')
print(test_column)
print(test_csv_data.shape)
print('train-----------')
print(train_column)
print(train_csv_data.shape)


sample_submission-----------
['id', 'hhb', 'hbo2', 'ca', 'na']
(10000, 5)
test-----------
['id', 'rho', '650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_src', '730_src', '740_src', '750_src', '760_src', '770_src', '780_src', '790_src', '800_src', '810_src', '820_src', '830_src', '840_src', '850_src', '860_src', '870_src', '880_src', '890_src', '900_src', '910_src', '920_src', '930_src', '940_src', '950_src', '960_src', '970_src', '980_src', '990_src', '650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst', '720_dst', '730_dst', '740_dst', '750_dst', '760_dst', '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst', '900_dst', '910_dst', '920_dst', '930_dst', '940_dst', '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']
(10000, 72)
train-----------
['id', 'rho', '650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_s

In [7]:
sample_data = pd.DataFrame(sample_csv_data, columns=sample_column)
sample_data.head()

,id,hhb,hbo2,ca,na
0,10000,0,0,0,0
1,10001,0,0,0,0
2,10002,0,0,0,0
3,10003,0,0,0,0
4,10004,0,0,0,0


In [8]:
test_data = pd.DataFrame(test_csv_data, columns=test_column)
print(test_data.shape)
test_data.head()

(10000, 72)


,id,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,...,900_dst,910_dst,920_dst,930_dst,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst
0,10000.0,15.0,0.15406,0.23275,0.30977,0.42949,0.51264,0.62558,0.74340,0.85418,...,0.000000e+00,0.000000e+00,1.432248e-14,0.000000e+00,0.000000e+00,6.332117e-15,1.429966e-14,0.000000e+00,NaN,7.320236e-14
1,10001.0,15.0,0.48552,0.56939,0.67575,0.79089,0.85114,0.92581,0.98071,0.98177,...,1.036013e-13,NaN,2.830975e-14,1.114337e-13,4.825731e-14,NaN,2.282485e-14,7.348414e-14,1.259055e-13,2.349874e-13
2,10002.0,10.0,0.46883,0.56085,0.62442,0.73172,0.81724,0.91517,0.94801,0.99108,...,NaN,NaN,NaN,NaN,1.569208e-11,6.242378e-12,NaN,1.219010e-11,NaN,NaN
3,10003.0,10.0,0.06905,0.07517,0.10226,0.14905,0.16182,0.19659,0.26085,0.36753,...,1.651177e-11,7.282747e-12,5.010879e-12,NaN,1.571023e-11,0.000000e+00,0.000000e+00,3.304247e-12,4.106134e-11,NaN
4,10004.0,25.0,0.00253,0.00757,0.01649,0.00128,0.00000,0.00000,0.00105,0.01975,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.732057e-17,4.110605e-17,0.000000e+00,0.000000e+00,1.910775e-16,2.215673e-15


In [9]:
train_data = pd.DataFrame(train_csv_data, columns=train_column)
print(train_data.shape)
train_data.head()

(10000, 76)


,id,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
0,0.0,25.0,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,...,NaN,0.000000e+00,NaN,1.067504e-18,5.998949e-18,4.378513e-17,5.59,4.32,8.92,4.29
1,1.0,10.0,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,...,1.343132e-08,6.112685e-09,2.130547e-09,NaN,9.710091e-09,NaN,0.00,2.83,7.25,4.64
2,2.0,25.0,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,NaN,10.64,3.00,8.40,5.16
3,3.0,10.0,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,...,2.245998e-10,1.299511e-10,7.782625e-11,NaN,4.088921e-10,NaN,5.67,4.01,5.05,4.35
4,4.0,15.0,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,...,1.457955e-13,8.769053e-14,NaN,1.330237e-13,NaN,NaN,11.97,4.41,10.78,2.42


In [10]:
#데이터 전처리 - 보간법
dst_columns = [k for k in train_data.columns if 'dst' in k]
train_dst = train_data[dst_columns]
test_dst = test_data[dst_columns]
train_data[dst_columns] = train_dst.interpolate(axis=1)
train_data.fillna(0, inplace=True)
test_data[dst_columns] = test_dst.interpolate(axis=1)
test_data.fillna(0, inplace=True)

In [11]:
src_list=['650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_src', '730_src', 
          '740_src', '750_src', '760_src', '770_src', '780_src', '790_src', '800_src', '810_src', '820_src', 
          '830_src', '840_src', '850_src', '860_src', '870_src', '880_src', '890_src', '900_src', '910_src', 
          '920_src', '930_src', '940_src', '950_src', '960_src', '970_src', '980_src', '990_src']
dst_list=['650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst', '720_dst', '730_dst', 
          '740_dst', '750_dst', '760_dst', '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', 
          '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst', '900_dst', '910_dst', 
          '920_dst', '930_dst', '940_dst', '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']

In [12]:
#for col in dst_list:
#    train_data[col] = train_data[col] * (train_data['rho'] ** 2)
#    test_data[col] = test_data[col] * (test_data['rho']**2)

In [13]:
#train_data = train_data.fillna(train_data.mean())

In [14]:
#test_data = test_data.fillna(test_data.mean())

In [15]:
print(np.isnan(train_data).values.any(),np.isnan(test_data).values.any())

False False


In [16]:
train_data

,id,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
0,0.0,25.0,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,...,3.227782e-19,0.000000e+00,5.337522e-19,1.067504e-18,5.998949e-18,4.378513e-17,5.59,4.32,8.92,4.29
1,1.0,10.0,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,...,1.343132e-08,6.112685e-09,2.130547e-09,5.920319e-09,9.710091e-09,9.710091e-09,0.00,2.83,7.25,4.64
2,2.0,25.0,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,1.329725e-18,10.64,3.00,8.40,5.16
3,3.0,10.0,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,...,2.245998e-10,1.299511e-10,7.782625e-11,2.433592e-10,4.088921e-10,4.088921e-10,5.67,4.01,5.05,4.35
4,4.0,15.0,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,...,1.457955e-13,8.769053e-14,1.103571e-13,1.330237e-13,1.330237e-13,1.330237e-13,11.97,4.41,10.78,2.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995.0,15.0,0.23929,0.30265,0.39929,0.51000,0.64072,0.77328,0.86722,0.95891,...,6.788642e-16,0.000000e+00,5.516467e-16,9.690979e-16,1.391635e-15,5.460702e-14,12.68,4.11,12.31,0.10
9996,9996.0,20.0,0.02583,0.00946,0.03650,0.01380,0.04093,0.04363,0.03260,0.05282,...,1.814274e-16,1.117148e-16,7.352416e-17,3.280294e-16,5.825347e-16,4.226436e-15,8.46,4.11,10.46,3.12
9997,9997.0,10.0,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,...,4.828451e-11,3.110419e-11,8.119095e-12,7.926532e-11,1.504115e-10,4.003657e-10,9.84,3.20,10.45,2.06
9998,9998.0,15.0,1.01477,1.01504,0.99125,0.98747,1.00717,1.01434,0.99529,1.01322,...,3.279271e-13,0.000000e+00,4.475946e-14,1.431808e-12,2.818857e-12,1.408742e-11,6.38,4.06,11.28,4.03


In [17]:
num_traindata = 8000
num_validdata = 2000
from sklearn.utils import shuffle
train_data = shuffle(train_data)
train_dataset = train_data[:num_traindata]
valid_dataset = train_data[num_traindata : num_traindata + num_validdata]
train_dataset = train_dataset.drop('id',axis = 1)
valid_dataset = valid_dataset.drop('id',axis = 1)
print(train_dataset.shape, valid_dataset.shape)

(8000, 75) (2000, 75)


In [18]:
#x_test, y_test 분류
x_test = valid_dataset.loc[:, '650_dst':'990_dst'] # test dataset
y_test = valid_dataset.loc[:, 'hhb':'na'] # test labels
x_test.shape, y_test.shape

((2000, 35), (2000, 4))

In [19]:
#sample Code 용
x_train_sample = train_data.loc[:, '650_dst':'990_dst'] # train dataset
y_train_sample = train_data.loc[:, 'hhb':'na'] # train labels
x_train_sample.shape, y_train_sample.shape

((10000, 35), (10000, 4))

In [20]:
#x_train, y_train 분류
x_train = train_dataset.loc[:, '650_dst':'990_dst'] # train dataset
y_train = train_dataset.loc[:, 'hhb':'na'] # train labels
x_train.shape, y_train.shape

((8000, 35), (8000, 4))

In [21]:
scaler = MinMaxScaler()
scaler.fit(x_train)
scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(x_test)

x_train = pd.DataFrame(scaled_x_train, columns = x_train.columns)
x_test = pd.DataFrame(scaled_x_test, index = x_test.index, columns = x_test.columns)
x_train.shape, x_test.shape

((8000, 35), (2000, 35))

In [22]:
scaler_sample = MinMaxScaler()
scaler_sample.fit(x_train_sample)

scaled_x_train_sample = scaler_sample.transform(x_train_sample)

x_train_sample = pd.DataFrame(scaled_x_train_sample, columns = x_train_sample.columns)
x_train_sample.shape

(10000, 35)

In [21]:
#Sample_code
avr_valid_mse = 0
def test_train_model(x_data, y_data, k=5):
    models = []
    
    k_fold = KFold(n_splits=k, shuffle=True, random_state=123)
    
    for train_idx, val_idx in k_fold.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_val, y_val = x_data.iloc[val_idx], y_data[val_idx]
    
        d_train = xgb.DMatrix(data = x_train, label = y_train)
        d_val = xgb.DMatrix(data = x_val, label = y_val)
        
        wlist = [(d_train, 'train'), (d_val, 'eval')]
        
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'seed':777
            }

        model = xgb.train(params=params, dtrain=d_train, num_boost_round=500, verbose_eval=500, evals=wlist)
        models.append(model)

    return models

models = {}
for label in y_train.columns:
    print(label)
    models[label] = test_train_model(x_train_sample, y_train_sample[label])
    

hhb
[0]	train-mae:5.34553	eval-mae:5.32861


KeyboardInterrupt: 

In [ ]:
#Sample_code

def test_train_model(x_data, y_data, k=5):
    models = []
    avr_valid_mse = 0
    count = 0
    k_fold = KFold(n_splits=k, shuffle=True, random_state=123)
    print('x_data shape : ',x_data.shape, 'y_data shape : ', y_data.shape)
    for train_idx, val_idx in k_fold.split(x_data):
        
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_val, y_val = x_data.iloc[val_idx], y_data[val_idx]
    

        model= MLPRegressor(hidden_layer_sizes = (256, 512, 512, 64), 
                            activation = 'relu', 
                            solver = 'adam', 
                            learning_rate = 'adaptive', 
                            alpha = 0.001, 
                            random_state = 123)
        
        model.fit(x_train, y_train)
        
        train_model = model.predict(x_train)
        test_model = model.predict(x_val)
        
        print('train_mse : ',np.sqrt(mean_squared_error(y_train,train_model)))
        print('train_score: ',r2_score(y_train, train_model))
        valid_mse = np.sqrt(mean_squared_error(y_val,test_model))
        avr_valid_mse += valid_mse
        count += 1
        print('** valid_mse : ',valid_mse)
        
        
        models.append(model)

    print('AVERAGE valide_mse : ' , (avr_valid_mse/count)) 
    print('--------------------------------------------------')
    return models

models = {}
for label in y_train.columns:
    print(label)
    models[label] = test_train_model(x_train_sample, y_train_sample[label])
    

hhb
x_data shape :  (10000, 35) y_data shape :  (10000,)
train_mse :  3.0380961521797762
train_score:  -0.024319587717075875
** valid_mse :  2.8948016613150243
train_mse :  2.974975579544387
train_score:  -0.002219524302518705
** valid_mse :  3.0526848404873483
train_mse :  2.9615082226528497
train_score:  -0.0014800551100981885
** valid_mse :  3.0272681403012167
train_mse :  2.9550532010224346
train_score:  0.00487636403753966
** valid_mse :  3.013961186581181
train_mse :  2.9528608829314615
train_score:  0.0034585585695300347
** valid_mse :  3.0269079925043676
AVERAGE valide_mse :  3.0031247642378274
--------------------------------------------------
hbo2
x_data shape :  (10000, 35) y_data shape :  (10000,)
train_mse :  0.9915800392567089
train_score:  0.0054889818791235
** valid_mse :  1.014036781065639
train_mse :  1.0029586069380299
train_score:  0.00042562347545083146
** valid_mse :  0.9888132802830619
train_mse :  0.998514777233219
train_score:  -0.0007966093312739009
** valid_m

In [197]:
#결과 출력
for col in models:
    print(col)
    preds = []
    for model in models[col]:
        preds.append(model.predict(test_data.loc[:, '650_dst':]))
    pred = np.mean(preds, axis=0)

    sample_data[col] = pred

hhb
hbo2
ca
na


In [198]:
sample_data.head()

,id,hhb,hbo2,ca,na
0,10000,8.229903,4.018909,9.130824,3.097383
1,10001,8.040452,4.012446,9.133354,3.085513
2,10002,7.927356,4.023058,9.107715,3.011358
3,10003,7.896996,4.025552,9.119115,3.022734
4,10004,8.250586,4.008378,9.015123,3.047657


In [ ]:
#Save Excel 
sample_data.to_csv('Dacon_Geonho_XGB_Boosting.csv', index=False)